In [ ]:
!pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.2/608.2 kB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.7.0.72
    Uninstalling opencv-python-headless-4.7.0.72:
      Successfully uninstalled opencv-python-headless-4.7.0.72


In [ ]:
!pip install sqlite-ulid sqlite-jsonschema sqlite-fastrand sqlite-xsv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.3 MB/s eta 0:00:00


In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')


/content/gdrive/MyDrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
%cd /mydrive/AI project

ln: failed to create symbolic link '/mydrive/My Drive': File exists
/content/gdrive/My Drive/AI project


In [ ]:
import cv2
import numpy as np
import os.path
import easyocr
from matplotlib import pyplot as plt
import sqlite3
import datetime

In [ ]:

classesFile = "./names.txt"
classNames = []

with open(classesFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
modelConfiguration = './yolov4-custom.cfg'
modelWeights = './yolov4-custom_best.weights'

net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [ ]:
def remove_special_chars(s):
    # make a array of char you want to remove
    special_chars = ['.', ',', '-', '_', '\\', '/',' ']
    for char in special_chars:
      # replce the char with '' 
        s = s.replace(char, '')
    return s

def most_frequent_string(arr):
    # Initialize a dictionary to count the frequency of each string
    freq = {}
    
    # Loop through the array and count the frequency of each string
    for string in arr:
        if string in freq:
            freq[string] += 1
        else:
            freq[string] = 1
    
    # Find the string with the highest frequency
    max_freq = 0
    most_freq_string = ""
    for string in freq:
        if freq[string] > max_freq:
            max_freq = freq[string]
            most_freq_string = string
    
    # Return the most frequent string
    return most_freq_string


In [ ]:
def findObjects(outputs, img,end):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []
    
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w, h = int(det[2]*wT), int(det[3]*hT)
                x, y = int((det[0]*wT) - w/2), int((det[1]*hT) - h/2)
                bbox.append([x, y, w, h])
                classIds.append(classId)
                confs.append(float(confidence))
    indices = cv2.dnn.NMSBoxes(bbox, confs, confThreshold, nmsThreshold)

    for i in indices:

        i = i
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        print(x,y,w,h)

        crop_img = img[y:y+h, x:x+w]
        print(len(crop_img))
        if (x<=0 or y<=0 or w<=0 or h<=0):
          print('crop_img is empty')
        else:
          gray_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
          reader = easyocr.Reader(['en'])
          result = reader.readtext(gray_img)
          if(result[0][1]==[]):
            print('empty')
            return result_rmsp,6
          else:
            result_rmsp=remove_special_chars(result[0][1])
            end=end+1
            print(end)
            print(result_rmsp)
            return result_rmsp,end

In [ ]:
whT = 320
confThreshold = 0.7
nmsThreshold = 0.3
task = 0
resultarr=[]

videopath = "./test_videos/v" + str(2) + ".mp4"
cap = cv2.VideoCapture(videopath)

while True:
    success, img = cap.read()
    blob = cv2.dnn.blobFromImage(
        img, 1/255, (whT, whT), [0, 0, 0], 1, crop=False)
    net.setInput(blob)
    layerNames = net.getLayerNames()

    outputNames = [layerNames[i-1] for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(outputNames)
    result,task = findObjects(outputs, img,task)
    resultarr.append(result)
    if(task==6):
      break
cap.release()

882 699 170 51
51
1
MH01TC0502
882 698 171 52
52
2
Mh01TC0502
857 703 170 56
56
3
MH01TC0502
840 703 166 61
61
4
MHOITC0502
779 705 182 69
69
5
MHOITC0502
742 711 188 72
72
6
MHOITC0502


In [ ]:
print(resultarr)
final_detected = most_frequent_string(resultarr)
current_time = datetime.datetime.now().strftime("%H:%M:%S")
print(final_detected)

['MH01TC0502', 'Mh01TC0502', 'MH01TC0502', 'MHOITC0502', 'MHOITC0502', 'MHOITC0502']
MHOITC0502


In [ ]:
# top_left = tuple(result[0][0][0])
# bottom_right = tuple(result[0][0][2])
# text = result[0][1]
# font = cv2.FONT_HERSHEY_SIMPLEX

In [ ]:
# img = cv2.imread(IMAGE_PATH)
# img = cv2.rectangle(img,top_left,bottom_right,(0,255,0),3)
# img = cv2.putText(img,text,top_left, font, 0.5,(255,255,255),2,cv2.LINE_AA)
# plt.imshow(img)
# plt.show()

In [ ]:
# img = cv2.imread(IMAGE_PATH)
# spacer = 140
# for detection in result: 
#     top_left = tuple(detection[0][0])
#     bottom_right = tuple(detection[0][2])
#     text = detection[1]
#     img = cv2.rectangle(img,top_left,bottom_right,(0,255,0),3)
#     img = cv2.putText(img,text,(10,spacer), font, 0.5,(0,255,0),2,cv2.LINE_AA)
#     spacer+=15
    
# plt.imshow(img)
# plt.show()

In [ ]:
cursor = sqlite3.connect('PMS.db')

In [ ]:
cursor.execute("create table Vehicle (Vehicle_No varchar(11) primary key , Owner_ID bigint not null , Owner_Type varchar(7) not null , Vehicle_Colour varchar(10) , Entry_Time time not null , Exit_Time time ) ;")

cursor.execute("create table Student (Student_ID bigint primary key , First_Name varchar(50) not null , Last_Name varchar(50) not null , DOB date not null , Gender varchar(20) not null , Contact_No bigint not null , Blood_Group varchar(3) , Year int not null, Vehicle_No varchar(11) not null, foreign key(Vehicle_No) references Vehicle(Vehicle_No) ) ;")
cursor.execute("create table Faculty (Faculty_ID bigint primary key , First_Name varchar(50) not null , Last_Name varchar(50) not null , DOB date not null , Gender varchar(20) not null , Contact_No bigint not null , Blood_Group varchar(3) , Department varchar(20) not null , Vehicle_No varchar(11) not null, foreign key(Vehicle_No) references Vehicle(Vehicle_No) ) ;")
cursor.execute("create table Staff (Staff_ID bigint primary key , First_Name varchar(50) not null , Last_Name varchar(50) not null , DOB date not null , Gender varchar(20) not null , Contact_No bigint not null , Blood_Group varchar(3) , Job varchar(20) not null , Vehicle_No varchar(11) not null, foreign key(Vehicle_No) references Vehicle(Vehicle_No) ) ;")
cursor.execute("create table Visitor (Visitor_ID bigint primary key , First_Name varchar(50) not null , Last_Name varchar(50) not null , Gender varchar(20) not null , Contact_No bigint not null , Blood_Group varchar(3) , Vehicle_No varchar(11) not null, foreign key(Vehicle_No) references Vehicle(Vehicle_No) ) ;")
# cursor.execute("create table Visitor (Visitor_ID bigint primary key , Vehicle_No varchar(10) not null, foreign key(Vehicle_No) references Vehicle(Vehicle_No) ) ;")

cursor.execute("create table Parking (Location_ID varchar(4) primary key , Location varchar(20) not null , No_Of_Parking_Spots int not null , Access_To varchar(7) not null ) ;")
# cursor.execute("create table Parking_Spot (Spot_ID int primary key , Location_ID varchar(4) not null , Vehicle_No varchar(10) not null, foreign key(Vehicle_No) references Vehicle(Vehicle_No) ) ;")

# cursor.execute("create table Violation_Index (Vehicle_No varchar(10) not null , Violation_Code int not null, foreign key(Vehicle_No) references Vehicle(Vehicle_No) ) ;")
# cursor.execute("create table Violation (Violation_Code int primary key , Description varchar(20) not null , fine int not null ) ;")

OperationalError: ignored

In [ ]:
# cursor.execute("create table Vehicle (Vehicle_No varchar(10) primary key , Owner_ID bigint not null , Owner_Type varchar(7) not null , Vehicle_Count int not null , Vehicle_Colour varchar(10) , Entry_Time time not null , Exit_Time time not null ) ;")

In [ ]:
# cursor.execute("drop table Vehicle ; " )

# cursor.execute("drop table Student ; " )
# cursor.execute("drop table Faculty ; " )
# cursor.execute("drop table Staff ; " )
# cursor.execute("drop table Visitor ; " )

# cursor.execute("drop table Parking ; " )
# cursor.execute("drop table Parking_Spot ; " )

# cursor.execute("drop table Violation_Index ; " )
# cursor.execute("drop table Violation ; " )

In [ ]:
final_detected = 55
# cursor.execute("INSERT INTO vehicle values('{}','{}','{}','{}','{}','{}');".format(final_detected , "2" , "A" , "B" , current_time, "" ))
# cursor.execute("UPDATE vehicle set Exit_Time = ",current_time,"where Vehicle_No = 55 ;")
cursor.execute("UPDATE vehicle SET Exit_Time = ? WHERE Vehicle_No = ?", (current_time, final_detected))
# cursor.execute("insert into Vehicle values ('CD23VF2145' , 2 , 'A' ,1, 'B' , '10:10:10' ) ; " )
# cursor.execute("insert into Vehicle values ('GH23PO2589' , 3 , 'A' , 1 , 'B' , '10:10:10' ) ; " )
# cursor.execute("insert into Vehicle values ('SW48DF1253' , 4 , 'A' , 1 , 'B' , '10:10:10' ) ; " )

In [ ]:
t1 = cursor.execute("select * from vehicle ;")
for row in t1:
  print(row)

('55', 2, 'A', 'B', '06:24:54', '06:36:42')
